<a href="https://colab.research.google.com/github/ShrushtiAsutkar/NNDL_2022/blob/master/6_RNN_LSTM_(milk_production_prediction).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RNN Model using LSTM to prediction milk production.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/content/monthly-milk-production-pounds.csv',index_col='Month',parse_dates=True)

In [ ]:
#Displaying month wise production data
df.index.freq='MS'

In [ ]:
df.head()

In [ ]:
#Displaying Data in Plot format
df.plot(figsize=(15,8))

In [ ]:
#provides classes and functions for the estimation of many different statistical models, as well as for conducting statistical tests, and statistical data exploration
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
#will show seperate graph for trend of production, seasonal changes observered and overall obseravation(same as above graph)
#The residual graph shows Overall - trend - seasonal, we can call it as noise and cannot we determined.
results = seasonal_decompose(df['Monthly milk production (pounds per cow)'])

In [ ]:
results.plot();

In [ ]:
len(df)

In [ ]:
train = df.iloc[:156]        # all data except the last 12 months data 168-12=156 to train the model.
test = df.iloc[156:]         # Last 12 months data used as testing set.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
#magnitude of the production values not in proper range
df.head(),df.tail()

In [ ]:
scaler.fit(train)
scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)

In [ ]:
scaled_train[:10]

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
# define generator
n_input = 3
n_features = 1
generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size=1)
#predicting 4th output using first three inputs
X,y = generator[0]
print(f'Given the Array: \n{X.flatten()}')
print(f'Predict this y: \n {y}')

In [ ]:
#predicting 4th output using first three inputs
X,y = generator[1]
print(f'Given the Array: \n{X.flatten()}')
print(f'Predict this y: \n {y}')

In [ ]:
# 1 row 3 columns and 1 feature
X.shape

In [ ]:
# We do the same thing, but now instead for 12 months
n_input = 12
generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size=1)
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
# define model
model = Sequential()   #layers are in sequence i.e one after the another
model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))   #100 neurons and ReLU activation function 
model.add(Dense(1))    #output layer
model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
# fit model
model.fit(generator,epochs=50)  #generator creates batches of 3-3 inputs and using that predict the next value

In [ ]:
loss_per_epoch = model.history.history['loss']
plt.plot(range(len(loss_per_epoch)),loss_per_epoch)
#we can obsreve the decrease in loss fro the graph over 50 epoch

In [ ]:
#taking last 12 months values in training set to make prediction for 1st value in test set
last_train_batch = scaled_train[-12:]
#reshaping data in 1, 3, 1 i.e 1, no. of input, no. of features
last_train_batch = last_train_batch.reshape((1, n_input, n_features))
model.predict(last_train_batch)

In [ ]:
scaled_test[0]     #original- 0.61  predicted- 0.67  difference is negligible

In [ ]:
#empty list of test predictions
test_predictions = []     
first_eval_batch = scaled_train[-n_input:]      #last 12 values from the training set
current_batch = first_eval_batch.reshape((1, n_input, n_features))   #reshaping this last 12 values

for i in range(len(test)):
    
    # get the prediction value for the first batch
    current_pred = model.predict(current_batch)[0]
    
    # append the prediction into the array
    test_predictions.append(current_pred) 
    
    # use the prediction to update the batch and remove the first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
test_predictions

In [ ]:
#changing back to original scale.
test.head()

In [ ]:
true_predictions = scaler.inverse_transform(test_predictions)
test['Predictions'] = true_predictions

In [ ]:
#Developed model is not 100% accurate, but is quite close 
test.plot(figsize=(15,8))

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse=sqrt(mean_squared_error(test['Monthly milk production (pounds per cow)'],test['Predictions']))
print(rmse)
#displays root mean square error.